<a href="https://colab.research.google.com/github/Marzieh777/Fraud-Detection-Crime-Rating/blob/master/fraud_detection_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gzip
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,accuracy_score,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support, roc_auc_score, average_precision_score)


from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
path_data= '/content/drive/My Drive/Colab Notebooks/fraud_prep.csv.gz'
fraud_df = pd.read_csv(path_data, compression='gzip', header=0, sep=',')



In [3]:
#NaN values for categorical and numerical:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

fraud_df_clean = DataFrameImputer().fit_transform(fraud_df)
print(len(fraud_df_clean))


284807


#Exploring data

In [4]:
CATEGORICAL_INPUT_FEATURES = ['V{}'.format(i) for i in range(30, 32)]
ALL_FEATURES = list(fraud_df_clean)

INPUT_FEATURES = ALL_FEATURES.copy()
INPUT_FEATURES.remove('Class') #target
INPUT_FEATURES.remove('V29') #seems not important
print(INPUT_FEATURES)
NUMERICAL_INPUT_FEATURES = [feature for feature in INPUT_FEATURES if feature not in CATEGORICAL_INPUT_FEATURES]
print(NUMERICAL_INPUT_FEATURES)
print(CATEGORICAL_INPUT_FEATURES)
fraud_df_clean.Class.value_counts()

['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'V30', 'V31']
['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
['V30', 'V31']


0    284315
1       492
Name: Class, dtype: int64

# Test train split

In [5]:
#splitting data into train and test:
from sklearn.model_selection import train_test_split
import numpy as np
#fraud_df_clean[fraud_df_clean==np.inf]=np.nan
#fraud_df_clean = fraud_df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
#fraud_df_clean = fraud_df_clean.dropna()

df_train, df_test = train_test_split(fraud_df_clean, test_size=0.3, random_state=1333)
X_train= df_train.drop('Class',axis =1)
X_test = df_test.drop('Class',axis =1)

y_train =df_train.Class
y_test =df_test.Class
print(len(y_train))
print(len(X_test))


199364
85443


#Encode categorical features v30, v31


In [23]:
# encode categorical attributes into a binary one-hot encoded representation in test data and train data
def encode_cat_features(cat_cols, train_data,test_data):
  train_encoded = pd.get_dummies(train_data[cat_cols], prefix_sep="__", columns=cat_cols)
  encoded_columns = list(train_encoded.columns[:])
  print(encoded_columns)
  cat_dummies = [col for col in train_encoded if "__" in col and col.split("__")[0] in cat_cols]
  test_encoded = pd.get_dummies(test_data[cat_cols], prefix_sep="__", columns=cat_cols)
  # Remove additional columns
  for col in test_encoded.columns:
    if ("__" in col) and (col.split("__")[0] in cat_cols) and col not in cat_dummies:
      print("Removing additional feature {}".format(col))
      test_encoded.drop(col, axis=1, inplace=True)
  #add missing cols
  for col in cat_dummies:
    if col not in test_encoded.columns:
      print("Adding missing feature {}".format(col))
      test_processed[col] = 0
  test_encoded = test_encoded[encoded_columns]
  return (train_encoded,test_encoded)

X_train_cat, X_test_cat = encode_cat_features(CATEGORICAL_INPUT_FEATURES,X_train, X_test)
X_train_num = X_train[NUMERICAL_INPUT_FEATURES]
X_test_num = X_test[NUMERICAL_INPUT_FEATURES]

df_train= pd.concat([X_train_cat,X_train_num, y_train], axis = 1)
df_test = pd.concat([X_test_cat, X_test_num, y_test], axis = 1)

df_train_rd = df_train.sample(frac =1)
df_test_rd= df_test.sample(frac = 1)

print(len(df_train_rd))
print(len(df_test_rd))

['V30__Abacus Federal Savings Bank', 'V30__Access Federal Credit Union', 'V30__Adirondack Bank', 'V30__Adirondack Trust Company, The', 'V30__Advantage Federal C.U.', 'V30__Alco Federal Credit Union', 'V30__Alden State Bank', 'V30__Alma Bank', 'V30__Alternatives Federal Credit Union', 'V30__Amalgamated Bank', 'V30__Amerasia Bank', 'V30__AmeriCU Credit Union', 'V30__American Broadcast Employees Federal Credit Union', 'V30__Apple Bank For Savings', 'V30__Atlas Bank', 'V30__Ballston Spa National Bank', 'V30__Banco Popular North America', 'V30__Bank of Akron', 'V30__Bank of America N.A. GA1-006-15-40', 'V30__Bank of Castile, The', 'V30__Bank of Cattaraugus', 'V30__Bank of Holland', 'V30__Bank of Hope', 'V30__Bank of Millbrook', 'V30__Bank of Richmondville', 'V30__Bank of Utica', 'V30__BankUnited, NA', 'V30__Bay Ridge Federal Credit Union', 'V30__Berkshire Bank (Pittsfield, MA)', 'V30__Berkshire Bank, The', 'V30__Bethpage Federal Credit Union', 'V30__Bridgehampton National Bank, The', 'V30__

# SMOTE

In [24]:
from imblearn.over_sampling import SMOTE
# SMOTE
X_train_rd =df_train_rd.drop('Class',axis = 1)
y_train_rd=df_train_rd.Class.values
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_sample(X_train_rd, y_train_rd)

X_test_rd =df_test_rd.drop('Class',axis = 1)
y_test_rd=df_test_rd.Class.values
cols = X_test_rd.columns

X_train_res = pd.DataFrame(X_train_res, columns= cols)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


#Performing XGboost Classifier

In [25]:
import xgboost as xgb

#Implementing XG-boost #max_depth= 20
clf = xgb.XGBClassifier(missing=9999999999,
                max_depth = 10,
                n_estimators=1000,
                learning_rate=0.1,
                nthread=4,
                subsample=1.0,
                colsample_bytree=0.5,
                min_child_weight = 3,
                seed=1301)

#eval_metric=aucpr
clf.fit(X_train_res, y_train_res, early_stopping_rounds=30, eval_metric="aucpr",
        eval_set=[ (X_test_rd, y_test_rd)])

print('Overall AUC TEST:', roc_auc_score(y_test_rd, clf.predict(X_test_rd, ntree_limit=clf.best_iteration))) 	
print('Overall AUC PR TEST:', average_precision_score(y_test_rd, clf.predict(X_test_rd, ntree_limit=clf.best_iteration))) 

print (classification_report(y_test_rd, clf.predict(X_test_rd)))



[0]	validation_0-aucpr:0.617606
Will train until validation_0-aucpr hasn't improved in 30 rounds.
[1]	validation_0-aucpr:0.663513
[2]	validation_0-aucpr:0.689048
[3]	validation_0-aucpr:0.717798
[4]	validation_0-aucpr:0.710419
[5]	validation_0-aucpr:0.703292
[6]	validation_0-aucpr:0.710724
[7]	validation_0-aucpr:0.723045
[8]	validation_0-aucpr:0.764259
[9]	validation_0-aucpr:0.755526
[10]	validation_0-aucpr:0.755443
[11]	validation_0-aucpr:0.75593
[12]	validation_0-aucpr:0.754216
[13]	validation_0-aucpr:0.753218
[14]	validation_0-aucpr:0.75307
[15]	validation_0-aucpr:0.751912
[16]	validation_0-aucpr:0.753374
[17]	validation_0-aucpr:0.75287
[18]	validation_0-aucpr:0.779573
[19]	validation_0-aucpr:0.797896
[20]	validation_0-aucpr:0.797479
[21]	validation_0-aucpr:0.803596
[22]	validation_0-aucpr:0.805274
[23]	validation_0-aucpr:0.805934
[24]	validation_0-aucpr:0.805017
[25]	validation_0-aucpr:0.804215
[26]	validation_0-aucpr:0.804478
[27]	validation_0-aucpr:0.802151
[28]	validation_0-aucpr